# DO NOT RUN THIS NOTEBOOK UNTIL THE PREVIOUS NOTEBOOK FINISHES COMPLETELY 

# THE PREVIOUS NOTEBOOK WILL TAKE 20-30 MINUTES.

# PLEASE BE PATIENT

In [ ]:
%%bash

### Source the environment
source ~/.bash_profile

AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please continue."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
        exit
    fi
else
  echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
  exit
fi

# DO NOT CONTINUE UNTIL THE CLUSTER STATUS IS `ACTIVE` ^^ ABOVE ^^.

# Associate IAM Policies with EKS Worker Nodes

In [ ]:
%%bash

### Source the environment
source ~/.bash_profile
    
AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
        aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .RoleName"

        aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .Arn" 

        export INSTANCE_ROLE_NAME=$(aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .RoleName")
        if [ -n "$INSTANCE_ROLE_NAME" ]; then
            echo "INSTANCE_ROLE_NAME:  $INSTANCE_ROLE_NAME"
            echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile
        else
            echo "Please re-run this notebook after cluster status:  ACTIVE."
        fi

        export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .Arn")
        if [ -n "$INSTANCE_PROFILE_ARN" ]; then
            echo "INSTANCE_PROFILE_ARN:  $INSTANCE_PROFILE_ARN"
            echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile
        else
            echo "Please re-run this notebook after cluster status:  ACTIVE."
        fi
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
        exit
    fi
else
  echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
  exit
fi

# Attach Roles
* Allow Access from/to the Elastic Container Registry (ECR)
* This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR. 
* We will load these custom Docker images in a later section.

In [ ]:
%%bash

source ~/.bash_profile

if [ -n "$INSTANCE_ROLE_NAME" ]; then
    echo "INSTANCE_ROLE_NAME:  $INSTANCE_ROLE_NAME"
    aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess
    echo "Completed"
else
    echo "Please re-run this notebook when the EKS cluster status is ACTIVE."
    exit
fi

# Associate IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# THIS WILL TAKE 5-10 MINUTES.  PLEASE BE PATIENT.

# _If you see `retryable error`s below, this is OK!_

In [ ]:
%%bash

### Source the environment
source ~/.bash_profile

AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
        eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve
        echo "Completed"
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
        exit
    fi
else
  echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
  exit
fi

# _If you see `retryable error`s ^^ above ^^, this is OK!_

In [ ]:
%%bash

### Source the environment
source ~/.bash_profile

AWS_OIDC_ISSUER=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text)
if [ -n "$AWS_OIDC_ISSUER" ]; then
    echo "AWS_OIDC_ISSUER:  $AWS_OIDC_ISSUER"
    echo "Completed"
else
    echo "AWS_OIDC_ISSUER:  $AWS_OIDC_ISSUER"
    echo "Something is wrong."
fi

# Update `~/.kube/config` with our new EKS cluster

In [ ]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

# Verify Your EKS Cluster


In [ ]:
%%bash

kubectl get ns


In [ ]:
%%bash

kubectl get nodes
